In [1]:
# -*- coding: utf-8 -*-
"""
Created on Thu Aug 13 16:05:25 2020

@author: Joukey
"""
import os
os.chdir('D:/EIIE_keras/')

from pgportfolio.marketdata.datamatrices import DataMatrices
from pgportfolio.tools.configprocess import load_config
import tensorflow as tf
import numpy as np
import tflearn

tf_config = tf.ConfigProto()
tf_config.gpu_options.per_process_gpu_memory_fraction = 0.2

np.random.seed(0)
tf.set_random_seed(0)

sess = tf.Session(config=tf_config)

C:\Users\Joukey\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Joukey\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Joukey\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Joukey\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: P

curses is not supported on this machine (please install/reinstall curses for an optimal experience)








In [2]:
#%% load config & data

config_2 = load_config(2)
data = DataMatrices.create_from_config(config_2)

D:\EIIE_keras\database\Data.db
1435680000 1498838400.0 1800 109 30 5e-05 poloniex 11 31 3 0.08 False False False


D:\EIIE_keras\pgportfolio\marketdata\datamatrices.py:55: FutureWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.

  features=type_list)
D:\EIIE_keras\pgportfolio\marketdata\globaldatamatrix.py:119: FutureWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.

  panel = panel_fillna(panel, "both")


it is so shit


In [3]:
#%% network structure

feature_number = config_2["input"]["feature_number"]
rows = config_2["input"]["coin_number"]
columns = config_2["input"]["window_size"]

input_tensor = tf.placeholder(tf.float32, shape=[None, feature_number, rows, columns]) #[None, 3, 11, 31]
previous_w = tf.placeholder(tf.float32, shape=[None, rows]) #[None, 11]
input_num = tf.placeholder(tf.int32, shape=[])
y = tf.placeholder(tf.float32, shape=[None, feature_number, rows])

def allint(l):
    return [int(i) for i in l]

def build_network(layer_config, input_tensor, previous_w, input_num, y):
    layer = layer_config['layers']
    

    
    network = tf.transpose(input_tensor, [0, 2, 3, 1])
    network = network / network[:, :, -1, 0, None, None]
    network = tf.layers.conv2d(network, 
                               filters = int(layer[0]["filter_number"]),
                               kernel_size = allint(layer[0]["filter_shape"]),
                               strides = allint(layer[0]["strides"]),
                               padding = layer[0]["padding"],
                               activation = layer[0]["activation_function"],
                               kernel_regularizer = layer[0]["regularizer"],
                               # weight_decay=layer[0]["weight_decay"],
                               )
    
    width = network.get_shape()[2]
    network = tf.layers.conv2d(network, 
                               filters = int(layer[1]["filter_number"]),
                               kernel_size = [1, width],
                               strides = [1, 1],
                               padding = "valid",
                               activation = layer[1]["activation_function"],
                               # kernel_regularizer=layer[1]["regularizer"],
                               kernel_regularizer='l2',
                               # weight_decay=layer[1]["weight_decay"],
                               )
    
    width = network.get_shape()[2]
    height = network.get_shape()[1]
    features = network.get_shape()[3]
    network = tf.reshape(network, [input_num, int(height), 1, int(width*features)])
    w = tf.reshape(previous_w, [-1, int(height), 1, 1])
    network = tf.concat([network, w], axis=3)
    network = tf.layers.conv2d(network, filters = 1, 
                               kernel_size = [1, 1], 
                               padding="valid",
                               # kernel_regularizer = layer[2]["regularizer"],
                               kernel_regularizer = 'l2'
                               # weight_decay=layer[2]["weight_decay"],
                               )
    network = network[:, :, 0, 0]
    btc_bias = tf.get_variable("btc_bias", [1, 1], dtype=tf.float32, initializer=tf.zeros_initializer)
    btc_bias = tf.tile(btc_bias, [input_num, 1])
    network = tf.concat([btc_bias, network], 1)
    voting = network
    output = tf.nn.softmax(network)
    return output
    
output = build_network(config_2, input_tensor, previous_w, input_num, y)
future_price = tf.concat([tf.ones([input_num, 1]), y[:, 0, :]], 1)
future_omega = (future_price * output)/tf.reduce_sum(future_price * output, axis=1)[:, None]

commission_ratio = config_2["trading"]["trading_consumption"]

w_t = future_omega[:input_num-1]  # rebalanced
w_t1 = output[1:input_num]
mu = 1 - tf.reduce_sum(tf.abs(w_t1[:, 1:]-w_t[:, 1:]), axis=1)*commission_ratio

pv_vector = tf.reduce_sum(output * future_price, reduction_indices=[1])*(tf.concat([tf.ones(1), mu], axis=0))
log_mean_free = tf.reduce_mean(tf.log(tf.reduce_sum(output * future_price, reduction_indices=[1])))
portfolio_value = tf.reduce_prod(pv_vector)
mean = tf.reduce_mean(pv_vector)
log_mean = tf.reduce_mean(tf.log(pv_vector))
standard_deviation = tf.sqrt(tf.reduce_mean((pv_vector - mean) ** 2))

sharp_ratio = (mean - 1) / standard_deviation
loss = -tf.reduce_mean(tf.log(pv_vector))

# regularization_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
# if regularization_losses:
#     for regularization_loss in regularization_losses:
#         loss += regularization_loss

global_step = tf.Variable(0, trainable=False)
learning_rate = config_2["training"]["learning_rate"]
decay_steps = config_2["training"]["decay_steps"]
decay_rate = config_2["training"]["decay_rate"]
learning_rate = tf.train.exponential_decay(learning_rate, global_step, decay_steps, decay_rate, staircase=True)
train_step = tf.train.AdamOptimizer(learning_rate).minimize(loss, global_step=global_step)


Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.


Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [4]:
#%%
# tf.reset_default_graph()
sess.run(tf.global_variables_initializer())


In [5]:
#%% training & logging & testing network

import numpy as np
import logging
import time

train_dir = "train_package"
s_path = "./" + train_dir + "/" + '2'+ "/netfile"
l_path = "./" + train_dir + "/" + '2' + "/tensorboard"

console_level = logging.INFO
logfile_level = logging.DEBUG

logging.basicConfig(filename=l_path.replace("tensorboard","programlog"), level=console_level)
console = logging.StreamHandler()
console.setLevel(console_level)
logging.getLogger().addHandler(console)


test_set = data.get_test_set()
train_set = data.get_training_set()
array = np.maximum.reduce(test_set['y'][:, 0, :], 1)
total = 1.0
for i in array:
    total = total * i

# logging.info("upper bound in test is %s" % total)
print("upper bound in test is %s" % total)
            
# init_tensor_board(log_file_dir) # tensorboard related func. omit now
starttime = time.time()

total_data_time = 0
total_training_time = 0
best_metric = 0
for i in range(config_2['training']["steps"]):
    step_start = time.time()
    
    batch = data.next_batch()
    batch_x = batch["X"]
    batch_y = batch["y"]
    batch_last_w = batch["last_w"]
    batch_setw = batch["setw"]

    finish_data = time.time()
    total_data_time += (finish_data - step_start)
    # self._agent.train(x, y, last_w=last_w, setw=setw)
    
    tflearn.is_training(True, sess)
    
    results = sess.run([train_step, output], feed_dict={input_tensor: batch_x, y: batch_y, previous_w: batch_last_w, input_num: batch_x.shape[0]})
    batch_setw(results[-1][:, 1:])
    
    total_training_time += time.time() - finish_data
    if i % 1000 == 0:
        print('hi~', i)
        # logging.info("average time for data accessing is %s"%(total_data_time/1000))
        # logging.info("average time for training is %s"%(total_training_time/1000))
        print("average time for data accessing is %s"%(total_data_time/1000))
        print("average time for training is %s"%(total_training_time/1000))
        total_training_time = 0
        total_data_time = 0
        # self.log_between_steps(i)
        tflearn.is_training(False, sess)
        
        batch_x = test_set["X"]
        batch_y = test_set["y"]
        batch_last_w = test_set["last_w"]
        batch_setw = test_set["setw"]
        
        v_pv, v_log_mean, v_loss, v_log_mean_free, weights = \
        sess.run([portfolio_value, log_mean, loss, log_mean_free, output], feed_dict={input_tensor: batch_x, y: batch_y, previous_w: batch_last_w, input_num: batch_x.shape[0]})
        
        # logging.info('='*30)
        # logging.info('step %d' % i)
        # logging.info('-'*30)
        print('='*30)
        print('step %d' % i)
        print('-'*30)
        
        batch_x = train_set["X"]
        batch_y = train_set["y"]
        batch_last_w = train_set["last_w"]
        batch_setw = train_set["setw"]
        
        loss_value = sess.run([loss], feed_dict={input_tensor: batch_x, y: batch_y, previous_w: batch_last_w, input_num: batch_x.shape[0]})
        # logging.info('training loss is %s\n' % loss_value)
        # logging.info('the portfolio value on test set is %s\nlog_mean is %s\n'
        #              'loss_value is %3f\nlog mean without commission fee is %3f\n' % \
        #              (v_pv, v_log_mean, v_loss, v_log_mean_free))
        # logging.info('='*30+"\n")
        
        print('training loss is %s\n' % loss_value)
        print('the portfolio value on test set is %s\nlog_mean is %s\n'
                     'loss_value is %3f\nlog mean without commission fee is %3f\n' % \
                     (v_pv, v_log_mean, v_loss, v_log_mean_free))
        print('='*30+"\n")
        
        if v_pv > best_metric:
            best_metric = v_pv
            # logging.info("get better model at %s steps,"
            #              " whose test portfolio value is %s" % (i, v_pv))
            print("get better model at %s steps,"
                         " whose test portfolio value is %s" % (i, v_pv))

batch_x = test_set["X"]
batch_y = test_set["y"]
batch_last_w = test_set["last_w"]
batch_setw = test_set["setw"]

pv, log_mean = sess.run([portfolio_value, log_mean], feed_dict={input_tensor: batch_x, y: batch_y, previous_w: batch_last_w, input_num: batch_x.shape[0]})

# logging.warning('the portfolio value train No.%s is %s log_mean is %s,'
#                 ' the training time is %d seconds' % ('2', pv, log_mean, time.time() - starttime))
print('the portfolio value train No.%s is %s log_mean is %s,'
                ' the training time is %d seconds' % ('2', pv, log_mean, time.time() - starttime))

# return self.__log_result_csv(index, time.time() - starttime)

upper bound in test is 1.3944850246036507e+30
hi~ 0
average time for data accessing is 1.4994144439697265e-06
average time for training is 0.0008523170948028564
step 0
------------------------------
training loss is [-2.3442582e-05]

the portfolio value on test set is 1.9160615
log_mean is 0.00023424672
loss_value is -0.000234
log mean without commission fee is 0.000260


get better model at 0 steps, whose test portfolio value is 1.9160615
hi~ 1000
average time for data accessing is 0.0014606683254241944
average time for training is 0.004001790761947632
step 1000
------------------------------
training loss is [-3.0979885e-05]

the portfolio value on test set is 3.2272246
log_mean is 0.00042205345
loss_value is -0.000422
log mean without commission fee is 0.000557


get better model at 1000 steps, whose test portfolio value is 3.2272246
hi~ 2000
average time for data accessing is 0.001424750566482544
average time for training is 0.0038971829414367674
step 2000
-------------------------

hi~ 18000
average time for data accessing is 0.001487295389175415
average time for training is 0.004119687080383301
step 18000
------------------------------
training loss is [-0.000118450145]

the portfolio value on test set is 112.238106
log_mean is 0.0017005126
loss_value is -0.001701
log mean without commission fee is 0.003331


get better model at 18000 steps, whose test portfolio value is 112.238106
hi~ 19000
average time for data accessing is 0.0015154640674591064
average time for training is 0.004163532495498657
step 19000
------------------------------
training loss is [-0.000118180615]

the portfolio value on test set is 107.70729
log_mean is 0.0016856685
loss_value is -0.001686
log mean without commission fee is 0.003299


hi~ 20000
average time for data accessing is 0.0014720189571380616
average time for training is 0.004037446737289428
step 20000
------------------------------
training loss is [-0.000118818345]

the portfolio value on test set is 112.041885
log_mean is 0.0

hi~ 38000
average time for data accessing is 0.001425569772720337
average time for training is 0.0039484605789184574
step 38000
------------------------------
training loss is [-0.00014403413]

the portfolio value on test set is 134.68246
log_mean is 0.0017661814
loss_value is -0.001766
log mean without commission fee is 0.003773


get better model at 38000 steps, whose test portfolio value is 134.68246
hi~ 39000
average time for data accessing is 0.0014941909313201905
average time for training is 0.004040781736373901
step 39000
------------------------------
training loss is [-0.00013555506]

the portfolio value on test set is 134.6653
log_mean is 0.0017661358
loss_value is -0.001766
log mean without commission fee is 0.003796


hi~ 40000
average time for data accessing is 0.0014544613361358644
average time for training is 0.004009496450424194
step 40000
------------------------------
training loss is [-0.00014402071]

the portfolio value on test set is 127.30019
log_mean is 0.0017458

hi~ 59000
average time for data accessing is 0.001526843786239624
average time for training is 0.004054634809494018
step 59000
------------------------------
training loss is [-0.00016928886]

the portfolio value on test set is 75.5433
log_mean is 0.0015578916
loss_value is -0.001558
log mean without commission fee is 0.003974


hi~ 60000
average time for data accessing is 0.0015588691234588623
average time for training is 0.004103624343872071
step 60000
------------------------------
training loss is [-0.00017048554]

the portfolio value on test set is 69.87894
log_mean is 0.0015298143
loss_value is -0.001530
log mean without commission fee is 0.003961


hi~ 61000
average time for data accessing is 0.001519916296005249
average time for training is 0.003973548650741577
step 61000
------------------------------
training loss is [-0.00017136711]

the portfolio value on test set is 73.42212
log_mean is 0.0015476315
loss_value is -0.001548
log mean without commission fee is 0.004000


hi~ 

the portfolio value train No.2 is 71.714745 log_mean is 0.0015391565, the training time is 449 seconds


In [6]:
48.46767
35.595684
38.275093

38.275093

In [9]:
test_set['X'].shape

(2776, 3, 11, 31)